In [1]:
from posixpath import abspath
import sys,os
from pathlib import Path
import tensorflow as tf
sys.path.insert(0,os.path.join('Code/Preprocess'))

import torch
import torch.nn as nn

import tensorflow as tf # Adding padding to our dataset (train and test models)
import numpy as np
from DataPreparation import dataPrepare

import warnings
warnings.filterwarnings("ignore")

is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [2]:
class Data:

    def __init__(self,path) -> None:
        self.path = path

    def getData(self):
        """Considers the path of the dataset which holds the sparse matrix. With the help of tensorflow we are
        creating the padding for our dataset (X_train and X_test).

        Returns:
            Tensor: Tensor data with padding and dataframe
        """
        fetchData = dataPrepare()
    
        X_train, X_test, y_train, y_test =  fetchData.Vectorizer(self.path)

        # Adding the padding to our sparse matrix
        X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train.todense())
        X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test.todense())

        # make training and test sets in torch
        x_train = torch.from_numpy(X_train).type(torch.Tensor)
        x_test = torch.from_numpy(X_test).type(torch.Tensor)
        y_train = torch.from_numpy(y_train.to_numpy()).type(torch.Tensor)
        y_test = torch.from_numpy(y_test.to_numpy()).type(torch.Tensor)
        
        print(f'Size of our y-train dataset is {y_train.size()} and X-train dataset is {x_train.size()}')

        return x_train,x_test,y_train,y_test

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        """LSTM model which takes the following input:
                - Input Dimension
                - Hidden Layers (Number)
                - Total Number of Layers
                - Ouput Diemension

        Args:
            input_dim (int): Length of the dataset
            hidden_dim (int): Total hidden layer
            num_layers (int): Total layers to be present
            output_dim (int): Number of output diemension layer
        """
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # https://discuss.pytorch.org/t/why-do-i-get-typeerror-expected-np-ndarray-got-numpy-ndarray-when-i-use-torch-from-numpy-function/37525/3
        x = torch.from_numpy(np.asarray(x)).type(torch.Tensor)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers,tuple(x.size())[0] , self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, tuple(x.size())[0], self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [4]:
class LSTMImplement:

    def __init__(self,path, hidden_dim, num_layers, output_dim) -> None:
        """A cosntructor which takes the path of the dataset, number of hidden dimensions, number of layers and output
        diemensions as our parameters. This will plot our accuracy plot and loss plot.

        Args:
            path (String): Path of our dataset
            hidden_dim (int): Total hidden layer
            num_layers (int): Total layers to be present
            output_dim (int): Number of output diemension layer
        """
        self.path = path
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim

    def run(self):
        fetchData = dataPrepare()
        X_train, X_test, y_train, y_test =  fetchData.Vectorizer(self.path)
        
        x_train,x_test,y_train,y_test = Data(self.path).getData()
        input_dimension = X_train.shape[1]
        model = LSTM(input_dim=input_dimension, hidden_dim=self.hidden_dim, output_dim=self.output_dim, num_layers=self.num_layers)

        loss_fn = torch.nn.MSELoss()
        # print(f"Type of X_train {type(X_train)}")
        optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
        print(model)
        print(len(list(model.parameters())))
        for i in range(len(list(model.parameters()))):
            print(list(model.parameters())[i].size())

        num_epochs = 1000
        loss_val  = np.zeros(num_epochs)
        acc_val  = np.zeros(num_epochs)

        for t in range(num_epochs):
            # Forward pass
            y_train_pred = model(x_train)

            loss = loss_fn(y_train_pred, y_train)
            if t % 10 == 0 and t !=0:
                print(f"Epoch {t} MSE is {loss.item()}")
            loss_val[t] = loss.item()
            
            pred = torch.max(y_train_pred, 1)[1].eq(y_train).sum()
            # pred = model_accuracy(y_train_pred,y)
            if t % 10 == 0 and t !=0:
                print(f"Epoch {t} accuracy(%) is {(100*pred/len(y_train)).item()}")
            
            # Zero out gradient, else they will accumulate between epochs
            optimiser.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimiser.step()

In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
    
    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

In [7]:
lst = LSTMImplement('YoutubeComplete.csv',28,4,1).run()

Size of our y-train dataset is torch.Size([952]) and X-train dataset is torch.Size([952, 1, 2701])
LSTM(
  (lstm): LSTM(2701, 28, num_layers=4, batch_first=True)
  (fc): Linear(in_features=28, out_features=1, bias=True)
)
18
torch.Size([112, 2701])
torch.Size([112, 28])
torch.Size([112])
torch.Size([112])
torch.Size([112, 28])
torch.Size([112, 28])
torch.Size([112])
torch.Size([112])
torch.Size([112, 28])
torch.Size([112, 28])
torch.Size([112])
torch.Size([112])
torch.Size([112, 28])
torch.Size([112, 28])
torch.Size([112])
torch.Size([112])
torch.Size([1, 28])
torch.Size([1])
Epoch 10 MSE is 0.2740251421928406
Epoch 10 accuracy(%) is 57.24789810180664
Epoch 20 MSE is 0.25053107738494873
Epoch 20 accuracy(%) is 57.24789810180664
Epoch 30 MSE is 0.24536658823490143
Epoch 30 accuracy(%) is 57.24789810180664
Epoch 40 MSE is 0.24475184082984924
Epoch 40 accuracy(%) is 57.24789810180664
Epoch 50 MSE is 0.24481554329395294
Epoch 50 accuracy(%) is 57.24789810180664
Epoch 60 MSE is 0.2448105663